# Load the data

In [22]:
from result_saver import SaverProvider
provider = SaverProvider()

In [23]:
from Scratch import metadata_loader

DEVICE = "ibm_sherbrooke"
DISTANCE = int(30)
ROUNDS = str(30) 
LOGICAL = str(0)

md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md.drop(columns=["sampled_state", "num_qubits", "code",  "job_status", "optimization_level", "extra", "tags_xp", "tags"])
md = md[md["backend_name"] == DEVICE]


md = md[md["distance"]==DISTANCE]
# md = md[md["rounds"]==ROUNDS]
md = md[md["logical"]==LOGICAL]

# md

In [24]:
unique_rounds = md['rounds'].unique().tolist()
unique_rounds = [int(r) for r in unique_rounds]
unique_rounds.sort(reverse=True)
unique_rounds

[5, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 30, 31, 33, 35]

In [33]:
from tqdm import tqdm
import numpy as np

memories_by_rounds = {rounds: [] for rounds in unique_rounds}
for index, row in tqdm(md.iterrows(), total=md.shape[0], desc="Retrieving memory data"):
    job_id = row['job_id']
    rounds = int(row['rounds'])
    
    # Retrieve the job's memory data. Assuming provider.retrieve_job() is a function that does this.
    memory_data = provider.retrieve_job(job_id).result().get_memory()
    
    # Append the memory data to the corresponding list in the dictionary
    memories_by_rounds[rounds].append(memory_data)

for rounds in tqdm(memories_by_rounds, desc="Concatenating memory data"):
    if len(memories_by_rounds[rounds]) > 1:
        # Concatenate arrays along the first axis
        memories_by_rounds[rounds] = np.concatenate(memories_by_rounds[rounds], axis=0)
    elif len(memories_by_rounds[rounds]) == 1:
        # Just take the single array out of the list
        memories_by_rounds[rounds] = memories_by_rounds[rounds][0]

print(memories_by_rounds.keys())
print(memories_by_rounds[35].shape)

Concatenating memory data: 100%|██████████| 19/19 [00:00<00:00, 505.87it/s]

dict_keys([5, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 30, 31, 33, 35])
(15391, 1045)


# Initialize the code

In [34]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

layout = get_repcode_layout(distance=DISTANCE, backend=provider.get_backend(DEVICE), _is_hex=True)
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, 'cn6hk3mxhnxg008djq30', 2, 0.3, other_date=None) # rndm first sherbrooke job

# Decode each round

In [39]:
import cpp_soft_info

p_data = 6.869e-3  # mean sherbrooke noise

# Initialize your dictionary to store results
results_dict_weighted = {rounds: [0, 0] for rounds in unique_rounds}
results_dict_flat_one = {rounds: [0, 0] for rounds in unique_rounds}
results_dict_flat_informed = {rounds: [0, 0] for rounds in unique_rounds}

for synd_rounds in tqdm(memories_by_rounds.keys()):
    circuit = stim.Circuit.generated("repetition_code:memory",
                                     distance=DISTANCE,
                                     rounds=synd_rounds,
                                     after_clifford_depolarization=0.1)
    model = circuit.detector_error_model(decompose_errors=True)
    matching = pymatching.Matching.from_detector_error_model(model)
    qubit_mapping = get_repcode_IQ_map(layout, synd_rounds)

    memory = memories_by_rounds[synd_rounds]

    num_errors_weighted  = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory, 
                                               synd_rounds, qubit_mapping, grid_dict, 
                                               processed_scaler_dict, p_data=p_data, 
                                               p_mixed=p_data/10,  
                                               common_measure=-1)
    
    num_errors_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory, 
                                                        synd_rounds, qubit_mapping, grid_dict, 
                                                        processed_scaler_dict)
    
    num_errors_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_data/1, p_data/10, common_measure=-1) 
                    
    
    # Store the results in the dictionary
    results_dict_weighted[synd_rounds][0] += num_errors_weighted
    results_dict_weighted[synd_rounds][1] += len(memory)
    results_dict_flat_one[synd_rounds][0] += num_errors_flat
    results_dict_flat_one[synd_rounds][1] += len(memory)
    results_dict_flat_informed[synd_rounds][0] += num_errors_informed
    results_dict_flat_informed[synd_rounds][1] += len(memory)

100%|██████████| 19/19 [03:45<00:00, 11.89s/it]


In [38]:
print(results_dict_weighted)
print(results_dict_flat_one)
print(results_dict_flat_informed)

{5: [0, 1111], 7: [0, 5872], 8: [0, 1111], 9: [0, 1111], 10: [0, 1111], 11: [0, 1111], 13: [0, 1111], 15: [2, 1111], 17: [0, 1111], 19: [1, 1111], 21: [0, 1111], 23: [0, 1111], 25: [2, 2222], 27: [3, 2222], 29: [2, 1111], 30: [20, 2222], 31: [2, 1111], 33: [8, 1111], 35: [118, 15391]}
{5: [0, 1111], 7: [0, 5872], 8: [0, 1111], 9: [0, 1111], 10: [0, 1111], 11: [0, 1111], 13: [1, 1111], 15: [0, 1111], 17: [1, 1111], 19: [3, 1111], 21: [8, 1111], 23: [6, 1111], 25: [28, 2222], 27: [37, 2222], 29: [11, 1111], 30: [109, 2222], 31: [11, 1111], 33: [37, 1111], 35: [803, 15391]}
{5: [0, 1111], 7: [0, 5872], 8: [0, 1111], 9: [0, 1111], 10: [0, 1111], 11: [0, 1111], 13: [1, 1111], 15: [0, 1111], 17: [1, 1111], 19: [3, 1111], 21: [8, 1111], 23: [6, 1111], 25: [28, 2222], 27: [37, 2222], 29: [11, 1111], 30: [109, 2222], 31: [11, 1111], 33: [37, 1111], 35: [803, 15391]}
